In [55]:
# Example of structured tool

from langchain.agents import initialize_agent, AgentType, load_tools
from langchain.llms import OpenAI
from langchain.pydantic_v1 import BaseModel
from pydantic.tools import parse_obj_as
from typing import Type, Optional, Any
from langchain.tools import StructuredTool
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)

class AdditionSchema(BaseModel):
    first_number: int
    second_number: int

class CustomAdditionTool(StructuredTool):
    name = "addition_calculator"
    description = "Useful when you want to add two numbers"
    args_schema: Type[BaseModel] = AdditionSchema

    def _run(self, *args: Any, run_manager: Optional[CallbackManagerForToolRun] = None, **kwargs: Any) -> Any:
        # Can do either of the two

        # input = parse_obj_as(AdditionSchema, kwargs)
        # return input.first_number + input.second_number
        
        return kwargs["first_number"] + kwargs["second_number"]

    async def _arun(self, *args: Any, run_manager: Optional[AsyncCallbackManagerForToolRun] = None, **kwargs: Any) -> Any:
        raise NotImplementedError("Not supported")

In [56]:
class CustomSubstractionTool(StructuredTool):
    name = "substraction_calculator"
    description = "Useful when you want to find different between two numbers"
    args_schema: Type[BaseModel] = AdditionSchema

    def _run(self, *args: Any, run_manager: Optional[CallbackManagerForToolRun] = None, **kwargs: Any) -> Any:
        input = parse_obj_as(AdditionSchema, kwargs)
        return input.first_number - input.second_number

    async def _arun(self, *args: Any, run_manager: Optional[AsyncCallbackManagerForToolRun] = None, **kwargs: Any) -> Any:
        raise NotImplementedError("Not supported")

In [57]:
llm = OpenAI()

tools = load_tools(["ddg-search"], llm=llm)
tools.append(CustomAdditionTool())
tools.append(CustomSubstractionTool())

agent_executor = initialize_agent(llm= llm, tools=tools, agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [58]:
agent_executor.invoke({"input": "Who is Joe Biden's wife? What is the sum of their ages?"})




> Entering new AgentExecutor chain...
Action:
```
{
  "action": "duckduckgo_search",
  "action_input": "Joe Biden's wife"
}
```


Observation: After taking the Oath of Office on January 20, 2021, Joseph R. Biden and his wife Dr. Jill Biden officially became the 46th President and First Lady of the United States. Joe Biden might be the Democratic nominee for president of the United States, but his wife, Jill Biden, comes with an equally impressive resume. As noted by the Los Angeles Times , prior to Joe's time in the White House, Jill already had two master's degrees and had worked for almost 25 years as a college professor. President Joe Biden is opening up about his love for his wife, Dr. Jill Biden, and his late first wife, Neilia Hunter Biden . During a candid conversation on Monday's episode of iHeartPodcast's On ... President Joe Biden was tested for the virus following his wife's positive test, but his results were negative. White House press secretary Karine Jean-Pierre said t

{'input': "Who is Joe Biden's wife? What is the sum of their ages?",
 'output': "Joe Biden's wife is Dr. Jill Biden and their combined age is 151."}

In [53]:
response = agent_executor.invoke({"input": "Who is President Donald Trump's wife? What is the difference of their ages?"})
print(response)




> Entering new AgentExecutor chain...


Action: 
```
{
  "action": "duckduckgo_search",
  "action_input": "Donald Trump's wife age difference"
}
```

Observation: President Donald Trump and first lady Melania's 24-year age difference isn't the largest the White House has ever seen. Among the many first couples in White House history, age gaps have spanned ... Donald Trump is 24 years older than Melina (his third wife). However, Melina is his youngest wife. Trump is three years older than his first wife, Ivana, and 17 years older than his second wife, Marla. There were a lot of rumors about Donald and Melina's marriage, plus the fact that both of them slept in different rooms. The age difference between former President Donald Trump and his wife, Melania Trump, is approximately 25 years, 10 months, and 12 days. While the significant gap has sparked public interest and debate, it's important to approach the topic with an open mind and consider the nuances of their relationship. How old are the Republicans running in the 2024 e